# RUN THIS AFTER CONCATENATING BERT OUTPUT TO CSV'S

In [4]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import datetime

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
MERGED_TWEET_CHUNK_READ_PATH = r"/scratch/st-tlemieux-1/lfrymire/BERT-input/FINAL-INPUTS/ALL-TWEETS/COV-CT-Tweets/" #this should be a folder containing three folders - one for each type of CT tweet (make sure it ends in a slash)

AGGREGATED_TWEET_CHUNK_SAVE_PATH = r"/scratch/st-tlemieux-1/lfrymire/BERT-OUPUT/"

In [2]:
def get_files_in_folder(folder):
    
    return [f for f in listdir(folder) if isfile(join(folder, f))]

In [ ]:
chunk_paths = get_files_in_folder(MERGED_TWEET_CHUNK_READ_PATH)

In [6]:
cols_to_use = [BERT_CT_OUTPUT_COL, BERT_GEN_OUTPUT_COL, BERT_NON_CT_OUTPUT_COL, sentence1, 'user_id',
               'CT Tweet (Dummy)','NON-COVID CT Tweet (Dummy)', 'date', 'retweet']

bool_cols = [BERT_OUTPUT_COL_NAME, sentence1, 'CT Tweet (Dummy)','NON-COVID CT Tweet (Dummy)', 'retweet']
int_cols = ['user_ID']
# str_cols = ['Cleaned Tweet']

# obj_cols = ['language']

bool_cols_dict = {col:np.bool_ for col in bool_cols}
int_cols_dict = {col:np.int0 for col in int_cols}
# str_cols_dict = {col:str for col in str_cols}
# obj_cols_dict = {col:str for col in obj_cols}

# datatypes = {**bool_cols_dict, **str_cols_dict, **obj_cols_dict}
# datatypes = {**bool_cols_dict, **obj_cols_dict}
datatypes = {**bool_cols_dict, **int_cols_dict}

NameError: name 'BERT_CT_OUTPUT_COL' is not defined

In [ ]:
def get_date(s):
    return datetime.date(int(s[0:4]), int(s[5:7]), int(s[8:10]))

In [ ]:
agg_dict = {BERT_CT_OUTPUT_COL,:'mean',
            BERT_GEN_OUTPUT_COL:'mean',
            BERT_NON_CT_OUTPUT_COL:'mean',
            'COVID-SPECIFIC CT Tweet (Dummy)':'mean',
            'CT Tweet (Dummy)':'mean',
            'NON-COVID CT Tweet (Dummy)':'mean',
            'retweet':'mean'}

In [ ]:
for i, chunk_path in enumerate(chunk_paths):
    
    chunk = pd.read_csv(fr"{MERGED_TWEET_CHUNK_READ_PATH}{chunk_path}", usecols=cols_to_use, dtype=datatypes, parse_dates=['date'], date_parser=get_date)
    
    chunk.rename(columns={'sentence1':'COVID-SPECIFIC CT Tweet (Dummy)'})
    
    # MIGHT NEED 'user_id' TO BE AN INDEX TOO
    collapsed_chunk = chunk.set_index('date').groupby([pd.Grouper(freq='1W'), 'user_id']).agg(agg_dict).droplevel('user_id').reset_index()
    
    collapsed_chunk.to_csv(fr"{AGGREGATED_TWEET_CHUNK_SAVE_PATH}Chunk-{i}.csv", index=False)